In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd



#Função para Renomear arquivo recentemente baixado
import os,glob
from shutil import move

def get_last_filename_and_rename(save_folder, new_filename):
    files = glob.glob(save_folder + '/*')
    max_file = max(files, key=os.path.getctime)
    filename = max_file.split("/")[-1].split(".")[0]
    new_path = max_file.replace(filename, new_filename)
    move(max_file, new_path)
    #os.rename(max_file, new_path)
    return new_path


servico = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=servico)

#acessar site
driver.get("https://www2.aneel.gov.br/aplicacoes_liferay/tarifa/")
time.sleep(2)

[WDM] - Downloading:  63%|██████▎   | 4.27M/6.78M [00:00<00:00, 15.6MB/s]

In [2]:
#Cria lista para armazenar links de downloads
links_download = []
lista_agentes = []

#Seleciona Categoria do Agente Todos
select_cat_agente = Select(driver.find_element(By.NAME,"CategoriaAgente"))
select_cat_agente.select_by_visible_text('Todos')
time.sleep(1)

select_agente = Select(driver.find_element(By.NAME,"Agentes"))
#Traz todas as opcoes de Agente
options_agente = [x.text for x in select_agente.options]
i=2

for element in options_agente[2:]:
    #Seleciona o Agente
    #select_agente.select_by_index(element)
    #select_agente[element]
    select_agente.select_by_index(i)
    i+=1
    time.sleep(1)

    try:
        #Seleciona Tipo de Processo
        select_tipo_processo = Select(driver.find_element(By.NAME,"TipoProcesso"))
        select_tipo_processo.select_by_visible_text('Todos')
        time.sleep(1)

        #Seleciona Ano
        select_ano = Select(driver.find_element(By.NAME,"Ano"))
        select_ano.select_by_visible_text('Todos')
        time.sleep(1)

        #Clica em Procurar
        driver.find_element(By.XPATH,'/html/body/table/tbody/tr[2]/td[5]/input').click()
        time.sleep(1)

        #Pegar o link de download do arquivo PCAT mais atualizado (estrutura tarifaria)
        link = driver.find_element(By.XPATH, '/html/body/div/table/tbody/tr[4]/td[7]/a').get_attribute('href')
    except:
        continue
    links_download.append(link)
    lista_agentes.append(element)
    
    #data = driver.find_element(By.XPATH,'/html/body/div/table/tbody/tr[4]/td[4]').get_attribute('td')
    #datas.append(data)
    


[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [02:06<00:00, 56.3kB/s]


KeyboardInterrupt: 

In [3]:
links_download[0]

'https://www2.aneel.gov.br/aplicacoes/tarifa/arquivo/7019%20PCAT%20AmE%202022%20V02.xlsx'

In [54]:
#abrir nova instancia do chrome
driver = webdriver.Chrome(service=servico)

#Dicionario Chave: Agente(concessionaria) e Valor: Link para download
agente_links = dict(zip(lista_agentes, links_download))

#Baixar os Arquivos e Renomear
for agente in agente_links:
    driver.get(agente_links[agente])
    time.sleep(4) #posteriormente implementar uma forma de ver se o download acabou para substituir o sleep
    try:
        get_last_filename_and_rename(r'C:\Users\iagom\Downloads', agente)
    except:
        time.sleep(4)
        get_last_filename_and_rename(r'C:\Users\iagom\Downloads', agente)
#for key in agente_links:
#    print(agente_links[key])
    





In [4]:
planilha = pd.ExcelFile('https://www2.aneel.gov.br/aplicacoes/tarifa/arquivo/7019%20PCAT%20AmE%202022%20V02.xlsx', engine = 'openpyxl')

In [8]:
df = pd.read_excel(planilha, 'TA - Aplicação', skiprows = 2)

In [57]:
concessionaria = lista_agentes

i=0

#tratar os dados

for element in concessionaria:   
    
    try:
        planilha = pd.ExcelFile(element+".xlsx", engine = 'openpyxl')
        df = pd.read_excel(planilha, 'TA - Aplicação', skiprows = 2)
    except:
        print("Tabela sem TA Aplicação:", element)
        continue
    #limpeza
    df.drop('Unnamed: 0', inplace=True, axis=1)
    
    #adiciona a coluna com o nome da concessionaria
    df.insert(0,'CONCESSIONARIA', element)

    #filtra apenas as colunas de interesse
    df2 = df.filter(items=['CONCESSIONARIA', 'SUBGRUPO', 'MODALIDADE', 'CLASSE', 'SUBCLASSE', 'POSTO', 'UNIDADE', 'Total TUSD', 'Total TE'])
    
    #concatena todas os DF
    if i == 0:
        df_geral = df2
        i = 1
    else:
        df_geral = pd.concat([df_geral, df2], ignore_index=True)

df_geral.to_csv('tarifas.csv', index=False)
#df_geral.to_json('tarifas.json', orient='index')

In [58]:
df_geral.to_json('tarifas.json', orient='index')